In [1]:
import os
import json
import pickle
from bisect import bisect_left, bisect_right
from datetime import datetime, timedelta
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
DATADIR = "hackathon_data" # "./data"
transactions_path = f"{DATADIR}/avk_hackathon_data_transactions.csv"
data_transactions = pd.read_csv(f"{DATADIR}/avk_hackathon_data_transactions.csv")

In [4]:
data_story_texts = pd.read_csv(f"{DATADIR}/avk_hackathon_data_story_texts.csv")

In [6]:
data_story_logs = pd.read_csv(f"{DATADIR}/avk_hackathon_data_story_logs.csv")

In [7]:
data_party_x_socdem = pd.read_csv(f"{DATADIR}/avk_hackathon_data_party_x_socdem.csv")

In [9]:
data_party_products = pd.read_csv(f"{DATADIR}/avk_hackathon_data_party_products.csv")

In [10]:
data_account_x_balance = pd.read_csv(f"{DATADIR}/avk_hackathon_data_account_x_balance.csv")

In [99]:
data_transactions.transaction_type_desc.value_counts()

Покупка            9157744
Платеж             1977602
Оплата услуг        581098
Снятие наличных     271173
Name: transaction_type_desc, dtype: int64

In [87]:
data_transactions.transaction_type_desc.value_counts().shape

(4,)

In [81]:
data_transactions[data_transactions.transaction_type_desc == 'Платеж']

,party_rk,account_rk,financial_account_type_cd,transaction_dttm,transaction_type_desc,transaction_amt_rur,merchant_rk,merchant_type,merchant_group_rk,category
15,33875,48319,2,2019-01-01,Платеж,212.00,1116811.0,1.0,NaN,NaN
73,11874,6050,1,2019-01-01,Платеж,5100.00,252998.0,381.0,NaN,Финансовые услуги
87,19228,18118,2,2019-01-01,Платеж,411.00,623502.0,1.0,NaN,NaN
174,39829,39964,2,2019-01-01,Платеж,8100.00,252998.0,381.0,NaN,Финансовые услуги
179,17438,16499,1,2019-01-01,Платеж,23533.00,252998.0,381.0,NaN,Финансовые услуги
...,...,...,...,...,...,...,...,...,...,...
11987595,16124,12498,2,2019-12-31,Платеж,31450.00,1296741.0,388.0,243.0,Финансовые услуги
11987596,75327,93459,2,2019-12-31,Платеж,25362.32,825683.0,1.0,NaN,NaN
11987603,76760,95933,2,2019-12-31,Платеж,679.00,3056721.0,1.0,NaN,NaN
11987608,26430,55477,2,2019-12-31,Платеж,1020.00,3055847.0,1.0,NaN,NaN


In [166]:
idx

Float64Index([286.0,   1.0, 330.0, 291.0, 381.0, 229.0, 297.0,   5.0, 328.0,
              336.0, 335.0, 380.0, 226.0, 388.0, 230.0, 284.0, 378.0, 274.0,
              313.0, 367.0, 285.0, 283.0, 309.0, 275.0, 332.0,   9.0, 296.0,
              326.0, 491.0, 346.0, 429.0, 329.0, 447.0,  10.0],
             dtype='float64')

In [165]:
data_transactions[data_transactions.merchant_type.isin(idx)][['merchant_type', 'category']].drop_duplicates()

,merchant_type,category
1,330.0,Фаст Фуд
2,291.0,Супермаркеты
4,286.0,Супермаркеты
9,367.0,Красота
10,5.0,NaN
11,336.0,Супермаркеты
12,346.0,Разные товары
13,226.0,Транспорт
14,332.0,NaN
15,1.0,NaN


In [162]:
idx = data_transactions.merchant_type.value_counts()[:10].index

In [97]:
data_transactions[data_transactions.transaction_type_desc == 'Платеж'].merchant_type.value_counts().index

Float64Index([1.0, 381.0, 388.0, 389.0, 13.0, 387.0, 2.0, 459.0, 246.0, 390.0], dtype='float64')

In [96]:
data_transactions[data_transactions.transaction_type_desc == 'Платеж'].category.value_counts()

Финансовые услуги    751895
Развлечения              47
Связь/Телеком            36
Name: category, dtype: int64

In [100]:
data_transactions[data_transactions.transaction_type_desc == 'Оплата услуг'].merchant_type.value_counts().index

Float64Index([5.0, 9.0, 10.0, 2.0, 3.0, 6.0, 11.0, 4.0, 7.0, 8.0, 13.0, 12.0], dtype='float64')

In [12]:
data_story_texts

,story_id,name,story_text
0,794,Профессия: свадебный организатор,"["""",""Как стать свадебным организатором"",""Это Л..."
1,821,Бизнес: магазин электроники,"["""",""Как открыть интернет-магазин электроники""..."
2,824,Как у них: Сингапур,"["""",""Жизнь в Сингапуре"",""Это Славина. Вместе с..."
3,839,Студии йоги,"["""",""Студии йоги"",""7 московских студий, где мо..."
4,848,Как получать больше кэшбэка,"["""",""Как получать больше кэшбэка"",""Подключите ..."
...,...,...,...
1626,732,Лайфхак по инвестициям,"["""",""Лайфхак в Тинькофф Инвестициях"",""Как не п..."
1627,733,Лайфхак инвестиций,"["""",""Лайфхак в Тинькофф Инвестициях"",""Как не п..."
1628,741,Ошибка в реквизитах,"["""",""Как вернуть деньги при ошибке в реквизита..."
1629,746,Как у них: Барселона,"["""",""Жизнь в Барселоне"",""Это Ольга. Шесть лет ..."


In [86]:
data_story_logs

,party_rk,date_time,story_id,category,event
0,64266,2019-12-01 13:05:31,3840,39,like
1,55721,2019-12-01 13:18:08,3857,39,like
2,27563,2019-12-01 13:25:40,3174,78,like
3,61223,2019-12-01 13:25:20,3857,39,like
4,61223,2019-12-01 13:29:05,3765,73,like
...,...,...,...,...,...
13815,19043,2019-12-31 22:52:40,4117,71,like
13816,19043,2019-12-31 22:53:13,4120,39,like
13817,10195,2019-12-31 23:32:50,4117,71,favorite
13818,10195,2019-12-31 23:33:32,4120,39,like


In [88]:
data_story_logs.event.value_counts()

like        5920
favorite    5028
dislike      763
14]          592
77]          551
153          208
180          108
177           99
176           72
54]           54
19]           53
174           49
164           48
175           48
48            45
160           38
163           33
161           31
170           25
159           18
101]          14
182            9
76             6
102]           5
55]            2
173            1
Name: event, dtype: int64

In [85]:
data_party_x_socdem

,party_rk,gender_cd,age,marital_status_desc,children_cnt,region_flg
0,61243,F,70.0,NaN,0,0
1,66535,F,25.0,NaN,0,0
2,83721,M,55.0,Женат/замужем,0,0
3,88238,F,35.0,NaN,0,0
4,57179,F,30.0,Женат/замужем,0,0
...,...,...,...,...,...,...
49995,54994,M,40.0,Женат/замужем,0,0
49996,63391,M,35.0,Холост/не замужем,0,0
49997,5418,F,55.0,Холост/не замужем,0,0
49998,50273,F,35.0,Женат/замужем,0,0


In [15]:
data_party_products # какие продукты - важно для совета

,party_rk,product1,product2,product3,product4,product5,product6,product7
0,74874,1,1,0,0,0,0,0
1,83618,0,0,1,1,1,0,0
2,73766,0,0,0,1,1,0,0
3,60218,0,1,0,0,0,0,0
4,47253,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...
49995,52276,0,1,0,0,1,0,0
49996,53681,0,1,0,0,0,0,0
49997,13947,0,1,0,1,1,0,1
49998,94411,0,1,0,0,0,0,0


In [38]:
data_account_x_balance[data_account_x_balance.party_rk == 13734]

,party_rk,account_rk,prev_month,cur_month,balance_chng
11832,13734,21396,2019-04-30,2019-05-31,0.0
27368,13734,21396,2019-03-31,2019-04-30,0.0
100648,13734,21396,2019-10-31,2019-11-30,0.0
116651,13734,21396,2019-08-31,2019-09-30,0.0
125370,13734,21396,2019-11-30,2019-12-31,0.0
159438,13734,21396,2019-02-28,2019-03-31,0.0
170859,13734,21396,2019-01-31,2019-02-28,0.0
238902,13734,21396,2019-05-31,2019-06-30,5000.0
359676,13734,21396,2018-12-31,2019-01-31,0.0
398629,13734,21396,2019-07-31,2019-08-31,0.0


In [28]:
data_account_x_balance # у кого тут наны


,party_rk,account_rk,prev_month,cur_month,balance_chng
0,51607,55309,2018-12-31,2019-01-31,15000.0
1,59074,70471,2018-12-31,2019-01-31,0.0
2,33941,33065,2018-12-31,2019-01-31,-15000.0
3,9336,8611,2018-12-31,2019-01-31,-50000.0
4,13958,8421,2018-12-31,2019-01-31,-20000.0
...,...,...,...,...,...
614429,13734,124222,NaN,NaN,NaN
614430,33502,125615,NaN,NaN,NaN
614431,8735,123871,NaN,NaN,NaN
614432,91302,122557,NaN,NaN,NaN


In [48]:
a = data_account_x_balance[data_account_x_balance.prev_month.isna()].party_rk

In [75]:
data_account_x_balance[data_account_x_balance.party_rk == a.values[3]]

,party_rk,account_rk,prev_month,cur_month,balance_chng
14367,59046,125919,NaN,NaN,NaN
605608,59046,123831,2019-11-30,2019-12-31,55000.0


In [101]:
data_transactions.head()

,party_rk,account_rk,financial_account_type_cd,transaction_dttm,transaction_type_desc,transaction_amt_rur,merchant_rk,merchant_type,merchant_group_rk,category
0,20337,19666,1,2019-01-01,Покупка,84.00,88676.0,348.0,NaN,Сувениры
1,63404,72991,1,2019-01-01,Покупка,410.00,887248.0,330.0,725.0,Фаст Фуд
2,24789,23517,2,2019-01-01,Покупка,701.44,830014.0,291.0,NaN,Супермаркеты
3,57970,64838,2,2019-01-01,Покупка,6203.70,363834.0,278.0,454.0,Дом/Ремонт
4,12232,11591,2,2019-01-01,Покупка,734.53,85919.0,286.0,878.0,Супермаркеты


In [141]:
from sklearn.model_selection import train_test_split

train_party, valid_party = train_test_split(
    pd.read_csv(transactions_path, usecols=['party_rk']).party_rk.unique(), 
    train_size=0.8, random_state=42
)

print(f'Train: {len(train_party)} Val: {len(valid_party)}')

Train: 40000 Val: 10000


In [174]:
data_transactions.sort_values(by=['party_rk', 'transaction_dttm'])[-40:]

,party_rk,account_rk,financial_account_type_cd,transaction_dttm,transaction_type_desc,transaction_amt_rur,merchant_rk,merchant_type,merchant_group_rk,category
8436462,94610,116467,2,2019-10-07,Покупка,226.52,842867.0,286.0,NaN,Супермаркеты
7609617,94610,116467,2,2019-10-08,Покупка,359.89,1967365.0,286.0,878.0,Супермаркеты
8638250,94610,116467,2,2019-10-08,Покупка,40.06,842867.0,286.0,NaN,Супермаркеты
8858794,94610,116467,2,2019-10-08,Покупка,2056.00,1929209.0,306.0,270.0,Одежда/Обувь
8595491,94610,116467,2,2019-10-10,Платеж,10100.00,624380.0,1.0,NaN,NaN
9633749,94610,116467,2,2019-10-10,Оплата услуг,61.00,362267.0,5.0,632.0,NaN
8198021,94610,116467,2,2019-10-11,Покупка,720.54,1967365.0,286.0,878.0,Супермаркеты
8263749,94610,116467,2,2019-10-11,Покупка,110.00,397616.0,330.0,NaN,Фаст Фуд
8664195,94610,116467,2,2019-10-11,Покупка,2420.00,1613334.0,309.0,7.0,Одежда/Обувь
8155585,94610,116467,2,2019-10-14,Покупка,263.01,842867.0,286.0,NaN,Супермаркеты


In [179]:
days = data_transactions.groupby('party_rk')['transaction_dttm'].apply(lambda x: len(np.unique(x))).sort_values()

In [180]:
import matplotlib.pyplot as plt
%matplotlib inline

In [184]:
ndays, counts = np.unique(days, return_counts=True)

In [185]:
for n,c in zip(ndays, counts):
    print(n, c)

2 3
3 1
4 5
5 3
6 12
7 22
8 22
9 50
10 88
11 103
12 120
13 158
14 205
15 225
16 295
17 311
18 348
19 402
20 463
21 500
22 506
23 546
24 592
25 685
26 662
27 685
28 679
29 679
30 628
31 621
32 645
33 588
34 560
35 561
36 556
37 546
38 540
39 522
40 520
41 503
42 536
43 435
44 497
45 443
46 433
47 447
48 445
49 406
50 422
51 393
52 384
53 379
54 364
55 370
56 336
57 362
58 373
59 335
60 324
61 337
62 306
63 372
64 295
65 318
66 275
67 287
68 289
69 284
70 274
71 287
72 262
73 306
74 240
75 237
76 242
77 261
78 244
79 245
80 231
81 198
82 216
83 249
84 207
85 187
86 230
87 203
88 206
89 223
90 184
91 220
92 177
93 189
94 196
95 194
96 173
97 161
98 195
99 147
100 167
101 167
102 177
103 160
104 162
105 165
106 158
107 151
108 141
109 172
110 163
111 156
112 149
113 145
114 138
115 155
116 149
117 136
118 136
119 139
120 119
121 108
122 142
123 135
124 126
125 156
126 113
127 105
128 109
129 124
130 123
131 120
132 124
133 113
134 115
135 103
136 138
137 97
138 104
139 95
140 125
141 108
1

In [167]:
part_train = data_transactions#[data_transactions.party_rk.isin(train_party)]
part_valid = data_transactions[data_transactions.party_rk.isin(valid_party)]

In [168]:
data_trans_train = part_train[pd.to_datetime(part_train.transaction_dttm) < pd.to_datetime('2019-11-01')]
data_trans_valid = part_valid[pd.to_datetime(part_valid.transaction_dttm) >= pd.to_datetime('2019-11-01')]

In [171]:
len(data_trans_train.party_rk.unique())

48421

In [149]:
mapper = data_trans_train.groupby(['party_rk', 'merchant_type'])['transaction_amt_rur'].mean()

In [151]:
X = mapper.reset_index()

In [152]:
X.rename(columns={'transaction_amt_rur': 'transaction_amt_rur_mean'}, inplace=True)

In [156]:
mapper = data_trans_train.groupby(['party_rk', 'merchant_type'])['transaction_amt_rur'].std()

In [157]:
_x = mapper.reset_index().rename(columns={'transaction_amt_rur': 'transaction_amt_rur_std'})
X = X.merge(_x, how='left', on=['party_rk', 'merchant_type'])

In [159]:
X[X.party_rk == 1]

,party_rk,merchant_type,transaction_amt_rur_mean,transaction_amt_rur_std
0,1,1.0,11516.400000,2494.957947
1,1,10.0,261.000000,NaN
2,1,243.0,368.000000,NaN
3,1,244.0,714.000000,0.707107
4,1,286.0,1033.109091,536.546888
5,1,292.0,450.300000,NaN
6,1,297.0,1029.790000,5.736122
7,1,308.0,2501.500000,641.345851
8,1,330.0,225.900000,252.688412
9,1,336.0,869.000000,NaN


In [139]:
data_transactions_train = data_transactions[pd.to_datetime(data_transactions.transaction_dttm) < pd.to_datetime('2019-11-01')]

In [140]:
data_transactions_valid = data_transactions[pd.to_datetime(data_transactions.transaction_dttm) >= pd.to_datetime('2019-11-01')]

In [138]:
pd.to_datetime(data_transactions.transaction_dttm) < pd.to_datetime('2019-11-01')

0            True
1            True
2            True
3            True
4            True
            ...  
11987612    False
11987613    False
11987614    False
11987615    False
11987616    False
Name: transaction_dttm, Length: 11987617, dtype: bool

In [112]:
mapper2 = data_transactions.groupby(['party_rk', 'merchant_type'])['transaction_amt_rur'].std()

In [113]:
X = mapper.reset_index()

In [117]:
X.rename(columns={'transaction_amt_rur': 'transaction_amt_rur_mean'}, inplace=True)

In [123]:
X.head(3)

,party_rk,merchant_type,transaction_amt_rur_mean
0,1,1.0,10072.333333
1,1,10.0,261.000000
2,1,243.0,368.000000


In [120]:
_x = mapper2.reset_index().rename(columns={'transaction_amt_rur': 'transaction_amt_rur_std'})

In [124]:
_x.head(3)

,party_rk,merchant_type,transaction_amt_rur_std
0,1,1.0,4223.098568
1,1,10.0,NaN
2,1,243.0,NaN


In [128]:
X = X.merge(_x, how='left', on=['party_rk', 'merchant_type'])

In [130]:
X.head()

,party_rk,merchant_type,transaction_amt_rur_mean,transaction_amt_rur_std
0,1,1.0,10072.333333,4223.098568
1,1,10.0,261.000000,NaN
2,1,243.0,368.000000,NaN
3,1,244.0,714.000000,0.707107
4,1,286.0,1033.109091,536.546888
